In [151]:
%pip install -U bokeh

  Using cached bokeh-3.1.1-py3-none-any.whl (8.3 MB)
  Attempting uninstall: bokeh
    Found existing installation: bokeh 2.3.2
    Uninstalling bokeh-2.3.2:
      Successfully uninstalled bokeh-2.3.2
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import math
import numpy as np
import bokeh
import xyzservices.providers as xyz
import json
from bokeh import events
from bokeh.io import show, save, output_file, output_notebook
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, save
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool, Div, Legend
from bokeh.models.callbacks import CustomJS
import pyproj
from pyproj import Transformer
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv("tstationinfo.csv")

In [3]:
def latlon_to_webmercator(lat, lon):
    R = 6378137  # Earth radius in meters
    x = R * math.radians(lon)
    y = R * math.log(math.tan(math.pi / 4 + math.radians(lat) / 2))
    return x, y

df[['long_merc', 'lat_merc']] = df.apply(lambda row: latlon_to_webmercator(row['latitude'], row['longitude']), axis=1, result_type='expand')

In [4]:
df.head()

,id,name,muni_name,muni_id,latitude,longitude,community_type_id,community_type_description,subcommunity_type_id,subcommunity_type_description,...,etod_sub3d,etod_total,etod_type,line_descr,ov_emppipe,ex_emppipe,ov_hupipe,ex_hupipe,long_merc,lat_merc
0,1,21 DRY DOCK AVE,BOSTON,35,42.344721,-71.031526,1,Inner Core,1,Metropolitan Core Communities,...,7,30,Transit-Related,Silver Line SL2,1435.678810,0.0000,0.0,0.0,-7.907193e+06,5.212758e+06
1,2,25 DRY DOCK AVE,BOSTON,35,42.344749,-71.029960,1,Inner Core,1,Metropolitan Core Communities,...,7,30,Transit-Related,Silver Line SL2,1330.224265,569.8125,0.0,0.0,-7.907019e+06,5.212762e+06
2,3,570 BROADWAY @ ELMWOOD AVE,REVERE,248,42.411900,-71.011100,1,Inner Core,1,Metropolitan Core Communities,...,8,27,Transit-Related,116,0.000000,0.0000,0.0,0.0,-7.904919e+06,5.222881e+06
3,4,633 PARK AVE,REVERE,248,42.414400,-71.028600,1,Inner Core,1,Metropolitan Core Communities,...,7,24,Transit-Related,111,0.000000,0.0000,0.0,0.0,-7.906868e+06,5.223258e+06
4,5,742 HYDE PARK AVE,BOSTON,35,42.275000,-71.120000,1,Inner Core,1,Metropolitan Core Communities,...,5,23,Transit-Related,32,0.000000,0.0000,0.0,0.0,-7.917042e+06,5.202263e+06


In [5]:
df['etod_qtas'].unique()

array([4, 2, 3, 1, 0, 5])

In [6]:
df[(df['etod_qtas']!=0)&(df['etod_qtci']!=0)&(df['etod_qabc']!=0)]

,id,name,muni_name,muni_id,latitude,longitude,community_type_id,community_type_description,subcommunity_type_id,subcommunity_type_description,...,etod_sub3d,etod_total,etod_type,line_descr,ov_emppipe,ex_emppipe,ov_hupipe,ex_hupipe,long_merc,lat_merc
0,1,21 DRY DOCK AVE,BOSTON,35,42.344721,-71.031526,1,Inner Core,1,Metropolitan Core Communities,...,7,30,Transit-Related,Silver Line SL2,1435.678810,0.000000,0.0,0.0,-7.907193e+06,5.212758e+06
1,2,25 DRY DOCK AVE,BOSTON,35,42.344749,-71.029960,1,Inner Core,1,Metropolitan Core Communities,...,7,30,Transit-Related,Silver Line SL2,1330.224265,569.812500,0.0,0.0,-7.907019e+06,5.212762e+06
2,3,570 BROADWAY @ ELMWOOD AVE,REVERE,248,42.411900,-71.011100,1,Inner Core,1,Metropolitan Core Communities,...,8,27,Transit-Related,116,0.000000,0.000000,0.0,0.0,-7.904919e+06,5.222881e+06
3,4,633 PARK AVE,REVERE,248,42.414400,-71.028600,1,Inner Core,1,Metropolitan Core Communities,...,7,24,Transit-Related,111,0.000000,0.000000,0.0,0.0,-7.906868e+06,5.223258e+06
4,5,742 HYDE PARK AVE,BOSTON,35,42.275000,-71.120000,1,Inner Core,1,Metropolitan Core Communities,...,5,23,Transit-Related,32,0.000000,0.000000,0.0,0.0,-7.917042e+06,5.202263e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,322,WORCESTER UNION STATION,WORCESTER,348,42.261796,-71.793881,2,Regional Urban Centers,9,Major Regional Urban Centers,...,8,29,Transit-Related,Framingham/Worcester Line,0.000000,0.000000,0.0,0.0,-7.992058e+06,5.200276e+06
322,323,WORCESTER SQUARE,BOSTON,35,42.337472,-71.075598,1,Inner Core,1,Metropolitan Core Communities,...,14,47,Transit-Oriented,"Silver Line SL5, Silver Line SL4",2073.607139,2010.670081,357.0,0.0,-7.912099e+06,5.211666e+06
323,324,WORLD TRADE CENTER,BOSTON,35,42.348355,-71.042695,1,Inner Core,1,Metropolitan Core Communities,...,10,35,Transit-Supportive,"Silver Line SL1, Silver Line SL2",16587.484440,1316.124242,4719.0,819.0,-7.908437e+06,5.213305e+06
324,325,WYOMING HILL,MELROSE,178,42.452027,-71.069313,1,Inner Core,3,Streetcar Suburbs,...,8,23,Transit-Related,Haverhill Line,3.726000,3.726000,44.0,12.0,-7.911400e+06,5.228933e+06


In [ ]:
transit_df = df[(df['etod_qtas']!=0)&(df['etod_qtci']!=0)&(df['etod_qabc']!=0)]
p = figure(x_range=(-7992739.4389570, -7848024.1009258), y_range=(5146011.6792828, 5281579.1845907),
           x_axis_type="mercator", y_axis_type="mercator", title="MBTA communities", tools="lasso_select, box_zoom, reset", x_axis_location="above",toolbar_location="below")
p.xaxis.axis_label = 'Longitude'
p.yaxis.axis_label = 'Latitude'
p.add_tile(get_provider(Vendors.OSM))

s1 = ColumnDataSource(data=dict(long_merc=transit_df['long_merc'].to_list(), 
                                lat_merc=transit_df['lat_merc'].to_list(),
                                name=transit_df['name'].to_list(),
                                etod_qtas=transit_df['etod_qtas'].to_list(),
                                etod_qtci=transit_df['etod_qtci'].to_list(),
                                etod_qabc=transit_df['etod_qabc'].to_list()))
p.scatter(x='long_merc', y='lat_merc', size=7,  source=s1, alpha=0.6, color='red')

s2 = ColumnDataSource(data=dict(name=transit_df['name'].to_list(), 
                                etod_qtas=transit_df['etod_qtas'].to_list(), 
                                etod_qtci=transit_df['etod_qtci'].to_list(), 
                                etod_qabc=transit_df['etod_qabc'].to_list()))

factors = ['etod_qtas', 'etod_qtci', 'etod_qabc']
p2 = figure(title='Transit Subscore', y_range=s2.data['name'], x_range = (0,15.5), tools="", x_axis_location="above",toolbar_location="below")

p2.xaxis.axis_label = 'Score'
[a,b,c] = p2.hbar_stack(factors, y='name', source=s2, color=['red','blue','green'])

legend = Legend(title='Quintile Ranking', items=[
    ("land area accessible via transit"   , [a]),
    ("frequency and utilization of transit" , [b]),
    ("percent of commuters using a mode other than driving alone" , [c]),
], location="top", glyph_width=5)
p2.add_layout(legend, 'below')

p.js_on_event(events.Reset, CustomJS(args=dict(s1=s1, s2=s2, p2=p2), 
    code="""
        console.log(s2.data.name.length, s2.data.etod_qtas.length, s2.data.etod_qtci.length, s2.data.etod_qabc.length);
        console.log("s2.data before update:", JSON.stringify(s2.data));
        const d1 = s1.data;
        let new_name = Array.from(d1.name);
        let new_etod_qtas = Array.from(d1.etod_qtas);
        let new_etod_qtci = Array.from(d1.etod_qtci);
        let new_etod_qabc = Array.from(d1.etod_qabc);

        console.log(new_name.length, new_etod_qtas.length, new_etod_qtci.length, new_etod_qabc.length);
        let new_data = {name: Array.from(new_name), etod_qtas: Array.from(new_etod_qtas), etod_qtci: Array.from(new_etod_qtci), etod_qabc: Array.from(new_etod_qabc)};
        p2.y_range.factors = Array.from(new_name);  // Ensuring bars match the y-axis order
        p2.y_range.change.emit();
        p2.reset.emit(); 

        s2.data = new_data;
        s2.change.emit();
        console.log("s2.data after update:", JSON.stringify(s2.data));
    """),
)


s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2, p2 = p2), 
    code="""
        console.log(s2.data.name.length, s2.data.etod_qtas.length, s2.data.etod_qtci.length, s2.data.etod_qabc.length);
        console.log("s2.data before update:", JSON.stringify(s2.data));
        const inds = cb_obj.indices;
        if (inds.length === 0) { return; }
        console.log('inds', inds);
        const d1 = s1.data;
        s2._length = inds.length;
        
        let new_name = [];
        let new_etod_qtas = [];
        let new_etod_qtci = [];
        let new_etod_qabc = [];

        for (var i = 0; i < inds.length; i++) {
            new_name.push(d1.name[inds[i]]);
            new_etod_qtas.push(d1.etod_qtas[inds[i]]);
            new_etod_qtci.push(d1.etod_qtci[inds[i]]);
            new_etod_qabc.push(d1.etod_qabc[inds[i]]);
        }
        console.log(new_name.length, new_etod_qtas.length, new_etod_qtci.length, new_etod_qabc.length);
        let new_data = {name: Array.from(new_name), etod_qtas: Array.from(new_etod_qtas), etod_qtci: Array.from(new_etod_qtci), etod_qabc: Array.from(new_etod_qabc)};
        console.log('a');
        
        p2.y_range.factors = Array.from(new_name);
        p2.y_range.change.emit();
        p2.reset.emit(); 
        
        s2.data.name = new_name;
        s2.data.etod_qtas = new_etod_qtas;
        s2.data.etod_qtci = new_etod_qtci;
        s2.data.etod_qabc = new_etod_qabc;
        console.log(s2);
        console.log(s2.data.name.length, s2.data.etod_qtas.length, s2.data.etod_qtci.length, s2.data.etod_qabc.length, s2._length);
        console.log('d');
        s2.change.emit();
        console.log("s2.data after update:", JSON.stringify(s2.data));
    """),
)

# Create centered layout
layout = row(p, p2)
save(layout, 'transit.html')

<ipython-input-46-df3223bff870>:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(layout, 'transit.html')
<ipython-input-46-df3223bff870>:123: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(layout, 'transit.html')


'/Users/emilygan/Documents/github/viz/In-Transit/fp2/transit.html'

In [47]:
# "(etod_q0car + etod_q25k + etod_qrent + etod_qaff)
orient_df = df[(df['etod_q0car']!=0)&(df['etod_q25k']!=0)&(df['etod_qrent']!=0)&(df['etod_qaff']!=0)]
p = figure(x_range=(-7992739.4389570, -7848024.1009258), y_range=(5146011.6792828, 5281579.1845907),
           x_axis_type="mercator", y_axis_type="mercator", title="MBTA communities", tools="lasso_select, box_zoom, reset", x_axis_location="above",toolbar_location="below")
p.xaxis.axis_label = 'Longitude'
p.yaxis.axis_label = 'Latitude'
p.add_tile(get_provider(Vendors.OSM))

s1 = ColumnDataSource(data=dict(long_merc=orient_df['long_merc'].to_list(), 
                                lat_merc=orient_df['lat_merc'].to_list(),
                                name=orient_df['name'].to_list(),
                                etod_q0car=orient_df['etod_q0car'].to_list(),
                                etod_q25k=orient_df['etod_q25k'].to_list(),
                                etod_qrent=orient_df['etod_qrent'].to_list(),
                                etod_qaff=orient_df['etod_qaff'].to_list()))
p.scatter(x='long_merc', y='lat_merc', size=7,  source=s1, alpha=0.6, color='red')

s2 = ColumnDataSource(data=dict(name=orient_df['name'].to_list(), 
                                etod_q0car=orient_df['etod_q0car'].to_list(), 
                                etod_q25k=orient_df['etod_q25k'].to_list(), 
                                etod_qrent=orient_df['etod_qrent'].to_list(), 
                                etod_qaff=orient_df['etod_qaff'].to_list()))

factors = ['etod_q0car', 'etod_q25k', 'etod_qrent', 'etod_qaff']
p2 = figure(title='Transit Subscore', y_range=s2.data['name'], x_range = (0,15.5), tools="", x_axis_location="above",toolbar_location="below")

p2.xaxis.axis_label = 'Score'
[a,b,c, d] = p2.hbar_stack(factors, y='name', source=s2, color=['red','blue','green', 'pink'])

legend = Legend(title='Quintile Ranking', items=[
    ("percent of households with 0 vehicles available"   , [a]),
    ("percent of households earning <25k/year" , [b]),
    ("renters as a share of all hosueholds" , [c]),
    ("estimated household transportation costs as a percent of regional median income" , [d]),
], location="top", glyph_width=5)
p2.add_layout(legend, 'below')

p.js_on_event(events.Reset, CustomJS(args=dict(s1=s1, s2=s2, p2=p2), 
    code="""
        console.log(s2.data.name.length, s2.data.etod_q0car.length, s2.data.etod_q25k.length, s2.data.etod_qrent.length);
        console.log("s2.data before update:", JSON.stringify(s2.data));
        const d1 = s1.data;
        let new_name = Array.from(d1.name);
        let new_etod_q0car = Array.from(d1.etod_q0car);
        let new_etod_q25k = Array.from(d1.etod_q25k);
        let new_etod_qrent = Array.from(d1.etod_qrent);
        let new_etod_qaff = Array.from(d1.etod_qaff);

        console.log(new_name.length, new_etod_q0car.length, new_etod_q25k.length, new_etod_qrent.length);
        let new_data = {name: Array.from(new_name), etod_q0car: Array.from(new_etod_q0car), etod_q25k: Array.from(new_etod_q25k), etod_qrent: Array.from(new_etod_qrent), etod_qaff: Array.from(new_etod_qaff)};
        p2.y_range.factors = Array.from(new_name);  // Ensuring bars match the y-axis order
        p2.y_range.change.emit();
        p2.reset.emit(); 

        s2.data = new_data;
        s2.change.emit();
        console.log("s2.data after update:", JSON.stringify(s2.data));
    """),
)


s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2, p2 = p2), 
    code="""
        console.log(s2.data.name.length, s2.data.etod_q0car.length, s2.data.etod_q25k.length, s2.data.etod_qrent.length);
        console.log("s2.data before update:", JSON.stringify(s2.data));
        const inds = cb_obj.indices;
        if (inds.length === 0) { return; }
        console.log('inds', inds);
        const d1 = s1.data;
        s2._length = inds.length;
        
        let new_name = [];
        let new_etod_q0car = [];
        let new_etod_q25k = [];
        let new_etod_qrent = [];
        let new_etod_qaff = [];

        for (var i = 0; i < inds.length; i++) {
            new_name.push(d1.name[inds[i]]);
            new_etod_q0car.push(d1.etod_q0car[inds[i]]);
            new_etod_q25k.push(d1.etod_q25k[inds[i]]);
            new_etod_qrent.push(d1.etod_qrent[inds[i]]);
            new_etod_qaff.push(d1.etod_qaff[inds[i]]);
        }
        console.log(new_name.length, new_etod_q0car.length, new_etod_q25k.length, new_etod_qrent.length);
        let new_data = {name: Array.from(new_name), etod_q0car: Array.from(new_etod_q0car), etod_q25k: Array.from(new_etod_q25k), etod_qrent: Array.from(new_etod_qrent), etod_qaff: Array.from(new_etod_qaff)};
        console.log('a');
        
        p2.y_range.factors = Array.from(new_name);
        p2.y_range.change.emit();
        p2.reset.emit(); 
        
        s2.data.name = new_name;
        s2.data.etod_q0car = new_etod_q0car;
        s2.data.etod_q25k = new_etod_q25k;
        s2.data.etod_qrent = new_etod_qrent;
        s2.data.etod_qaff = new_etod_qaff;
        console.log(s2);
        console.log(s2.data.name.length, s2.data.etod_q0car.length, s2.data.etod_q25k.length, s2.data.etod_qrent.length, s2._length);
        console.log('d');
        s2.change.emit();
        console.log("s2.data after update:", JSON.stringify(s2.data));
    """),
)

# Create centered layout
layout = row(p, p2)
save(layout, 'orientation.html')

<ipython-input-47-23f237c503dd>:108: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(layout, 'orientation.html')
<ipython-input-47-23f237c503dd>:108: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(layout, 'orientation.html')


'/Users/emilygan/Documents/github/viz/In-Transit/fp2/orientation.html'

In [48]:
# etod_qwalk + etod_qdens + etod_qgrav
dev_df = df[(df['etod_qwalk']!=0)&(df['etod_qdens']!=0)&(df['etod_qgrav']!=0)]
p = figure(x_range=(-7992739.4389570, -7848024.1009258), y_range=(5146011.6792828, 5281579.1845907),
           x_axis_type="mercator", y_axis_type="mercator", title="MBTA communities", tools="lasso_select, box_zoom, reset", x_axis_location="above",toolbar_location="below")
p.xaxis.axis_label = 'Longitude'
p.yaxis.axis_label = 'Latitude'
p.add_tile(get_provider(Vendors.OSM))

s1 = ColumnDataSource(data=dict(long_merc=dev_df['long_merc'].to_list(), 
                                lat_merc=dev_df['lat_merc'].to_list(),
                                name=dev_df['name'].to_list(),
                                etod_qwalk=dev_df['etod_qwalk'].to_list(),
                                etod_qdens=dev_df['etod_qdens'].to_list(),
                                etod_qgrav=dev_df['etod_qgrav'].to_list()))
p.scatter(x='long_merc', y='lat_merc', size=7,  source=s1, alpha=0.6, color='red')

s2 = ColumnDataSource(data=dict(name=dev_df['name'].to_list(), 
                                etod_qwalk=dev_df['etod_qwalk'].to_list(), 
                                etod_qdens=dev_df['etod_qdens'].to_list(), 
                                etod_qgrav=dev_df['etod_qgrav'].to_list()))

factors = ['etod_qwalk', 'etod_qdens', 'etod_qgrav']
p2 = figure(title='Transit Subscore', y_range=s2.data['name'], x_range = (0,15.5), tools="", x_axis_location="above",toolbar_location="below")

p2.xaxis.axis_label = 'Score'
[a,b,c] = p2.hbar_stack(factors, y='name', source=s2, color=['red','blue','green'])

legend = Legend(title='Quintile Ranking', items=[
    ("walk score at transit station"   , [a]),
    ("residential density in station area" , [b]),
    ("employment proximity to station area" , [c]),
], location="top", glyph_width=5)
p2.add_layout(legend, 'below')

p.js_on_event(events.Reset, CustomJS(args=dict(s1=s1, s2=s2, p2=p2), 
    code="""
        console.log(s2.data.name.length, s2.data.etod_qwalk.length, s2.data.etod_qdens.length, s2.data.etod_qgrav.length);
        console.log("s2.data before update:", JSON.stringify(s2.data));
        const d1 = s1.data;
        let new_name = Array.from(d1.name);
        let new_etod_qwalk = Array.from(d1.etod_qwalk);
        let new_etod_qdens = Array.from(d1.etod_qdens);
        let new_etod_qgrav = Array.from(d1.etod_qgrav);

        console.log(new_name.length, new_etod_qwalk.length, new_etod_qdens.length, new_etod_qgrav.length);
        let new_data = {name: Array.from(new_name), etod_qwalk: Array.from(new_etod_qwalk), etod_qdens: Array.from(new_etod_qdens), etod_qgrav: Array.from(new_etod_qgrav)};
        p2.y_range.factors = Array.from(new_name);  // Ensuring bars match the y-axis order
        p2.y_range.change.emit();
        p2.reset.emit(); 

        s2.data = new_data;
        s2.change.emit();
        console.log("s2.data after update:", JSON.stringify(s2.data));
    """),
)


s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2, p2 = p2), 
    code="""
        console.log(s2.data.name.length, s2.data.etod_qwalk.length, s2.data.etod_qdens.length, s2.data.etod_qgrav.length);
        console.log("s2.data before update:", JSON.stringify(s2.data));
        const inds = cb_obj.indices;
        if (inds.length === 0) { return; }
        console.log('inds', inds);
        const d1 = s1.data;
        s2._length = inds.length;
        
        let new_name = [];
        let new_etod_qwalk = [];
        let new_etod_qdens = [];
        let new_etod_qgrav = [];

        for (var i = 0; i < inds.length; i++) {
            new_name.push(d1.name[inds[i]]);
            new_etod_qwalk.push(d1.etod_qwalk[inds[i]]);
            new_etod_qdens.push(d1.etod_qdens[inds[i]]);
            new_etod_qgrav.push(d1.etod_qgrav[inds[i]]);
        }
        console.log(new_name.length, new_etod_qwalk.length, new_etod_qdens.length, new_etod_qgrav.length);
        let new_data = {name: Array.from(new_name), etod_qwalk: Array.from(new_etod_qwalk), etod_qdens: Array.from(new_etod_qdens), etod_qgrav: Array.from(new_etod_qgrav)};
        console.log('a');
        
        p2.y_range.factors = Array.from(new_name);
        p2.y_range.change.emit();
        p2.reset.emit(); 
        
        s2.data.name = new_name;
        s2.data.etod_qwalk = new_etod_qwalk;
        s2.data.etod_qdens = new_etod_qdens;
        s2.data.etod_qgrav = new_etod_qgrav;
        console.log(s2);
        console.log(s2.data.name.length, s2.data.etod_qwalk.length, s2.data.etod_qdens.length, s2.data.etod_qgrav.length, s2._length);
        console.log('d');
        s2.change.emit();
        console.log("s2.data after update:", JSON.stringify(s2.data));
    """),
)

# Create centered layout
layout = row(p, p2)
save(layout, 'development.html')

<ipython-input-48-63acbd33b853>:101: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(layout, 'development.html')
<ipython-input-48-63acbd33b853>:101: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(layout, 'development.html')


'/Users/emilygan/Documents/github/viz/In-Transit/fp2/development.html'